In [13]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import glob
import random
import PIL

    import sklearn
    import sklearn.svm
    import sklearn.model_selection
    from sklearn.model_selection import train_test_split


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

%matplotlib inline
%load_ext autoreload
%autoreload 2

PyTorch Version:  1.7.1+cpu
Torchvision Version:  0.8.2+cpu
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate Positives and Negatives

In [17]:
from shutil import copyfile, copy 

# positives
broden_dir = 'broden_unmasked'
concept_dir = 'concept_data'
for b_concept in os.listdir(broden_dir):
    print(b_concept)
    dest_dir = os.path.join(concept_dir, b_concept, 'positives')
    if os.path.exists(dest_dir):
        continue
    os.makedirs(dest_dir) 
    for img in os.listdir(os.path.join(broden_dir, b_concept))[:100]:
        filepath = os.path.join(broden_dir, b_concept, img)
        copy(filepath, dest_dir)

airplane
air_conditioner
apron
arm
armchair
ashcan
awning
back
back_pillow
bag
balcony
bannister
base
basket
bathroom_s
bathtub
beak
bed
bedclothes
bedroom_s
bench
bicycle
bird
blind
blotchy
board
body
book
bookcase
bottle
bowl
box
brick
bridge
bucket
building
bumper
bus
bush
cabinet
can
candlestick
canopy
cap
car
cardboard
carpet
cat
ceiling
ceramic
chain_wheel
chair
chandelier
chest_of_drawers
chimney
clock
coach
coffee_table
column
computer
concrete
counter
countertop
cow
cup
curtain
cushion
desk
dining_room_s
dog
door
doorframe
door_frame
double_door
drawer
drinking_glass
ear
earth
engine
exhaust_hood
eye
eyebrow
fabric
fan
faucet
fence
field
figurine
fireplace
flag
floor
flower
flowerpot
fluorescent
food
foot
footboard
frame
glass
granite
grass
ground
hair
hand
handle
handle_bar
head
headboard
headlight
hill
horse
house
inside_arm
jar
keyboard
knob
laminate
lamp
leather
leg
lid
light
loudspeaker
manhole
metal
microwave
minibike
mirror
motorbike
mountain
mouse
mouth
muzzle
napkin
n

In [7]:
from shutil import copyfile, copy 

# negatives
broden_dir = 'broden_unmasked'
concept_dir = 'concept_data'
for b_concept in os.listdir(broden_dir):
    print(b_concept)
    dest_dir = os.path.join(concept_dir, b_concept, 'negatives')
    if os.path.exists(dest_dir):
        continue    
    os.makedirs(dest_dir) 
    count = 0
    for other_concepts in os.listdir(broden_dir):
        print("*", other_concepts)
        if not(other_concepts == b_concept):
            for img in os.listdir(os.path.join(broden_dir, other_concepts))[:5]:
                filepath = os.path.join(broden_dir, other_concepts, img)
                copy(filepath, dest_dir)
                count += 1
                if count > 100:
                    break
        if count > 100:
            break

airplane
air_conditioner
apron
arm
armchair
ashcan
awning
back
back_pillow
bag
balcony
bannister
base
basket
bathroom_s
bathtub
beak
bed
bedclothes
bedroom_s
bench
bicycle
bird
blind
blotchy
board
boat
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* back
* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
body
book
bookcase
bottle
bowl
box
brick
bridge
bucket
building
bumper
bus
bush
cabinet
can
candlestick
canopy
cap
car
cardboard
carpet
cat
ceiling
ceramic
chain_wheel
chair
chandelier
chest_of_drawers
chimney
clock
coach
coffee_table
column
computer
concrete
counter
countertop
cow
cup
curtain
cushion
desk
dining_room_s
dog
door
doorframe
door_frame
double_door
drawer
drinking_glass
ear
earth
engine
exhaust_hood
eye
eyebrow
fabric
fan
faucet
fence
field
figurine
fireplace
flag
floor
flower
flowerpot
fluorescent
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* b

* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
motorbike
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* back
* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
mountain
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* back
* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
mouse
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* back
* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
mouth
* airplane
* air_conditioner
* apron
* arm
* armchair
* ashcan
* awning
* back
* back_pillow
* bag
* balcony
* bannister
* base
* basket
* bathroom_s
* bathtub
* beak
* bed
* bedclothes
* bedroom_s
* bench
muzzle
* airplane


# Learn Concepts

In [2]:
import pickle
concept_dict = pickle.load(open('sq_concepts_102.pkl', 'rb'))

In [3]:
model_name = "squeezenet"
num_classes = 1000
batch_size = 8
num_epochs = 5
feature_extract = True

# Initialize the model for this run
model_ft = torch.hub.load('pytorch/vision:v0.6.0', 'squeezenet1_0', pretrained=True).eval()
input_size = 224

Using cache found in C:\Users\islam/.cache\torch\hub\pytorch_vision_v0.6.0


In [4]:
data_transforms = {
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Initialized Datasets and Dataloaders")

Initialized Datasets and Dataloaders


In [5]:
class SqueezenetBottom(nn.Module):
    def __init__(self, original_model):
        super(SqueezenetBottom, self).__init__()
        self.features = nn.Sequential(*list(list(original_model.children())[0].children())[:15], nn.Flatten())
        
    def forward(self, x):
        x = self.features(x)
        return x

class SqueezenetTop(nn.Module):
    def __init__(self, original_model):
        super(SqueezenetTop, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[1])
        
    def forward(self, x):
        x = x.view((-1, 512, 13, 13))
        x = self.features(x)
        x = x.view((-1, 1000))
        x = nn.Softmax(dim=-1)(x)
        return x

def get_model_parts(model):    
    model_bottom = SqueezenetBottom(model)
    model_top = SqueezenetTop(model)
    return model_bottom, model_top

model_bottom, model_top = get_model_parts(model_ft)
model_bottom.eval()
model_top.eval();

In [31]:
concept_dir = "concept_data/"
transform = data_transforms['val']

# iterate through each folder
for concept in os.listdir(concept_dir):
    print(concept)
    if concept in concept_dict:
        continue
        
    all_embeddings = []
    all_labels = []

    image_dataset = datasets.ImageFolder(os.path.join(concept_dir,concept), transform)
    dataloaders = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

    # load all of the images, get the embeddings
    for inputs, labels in dataloaders:
        embeddings = model_bottom(inputs)
        all_embeddings.extend(embeddings.detach().numpy())
        all_labels.extend(labels.detach().numpy())

    # train an svm on the pos, neg
    X_train, X_test, y_train, y_test = train_test_split(all_embeddings, all_labels)
    svm = sklearn.svm.SVC(kernel="linear", C=0.0001, probability=True)
    svm.fit(X_train, y_train)
    train_acc = svm.score(X_train, y_train)
    test_acc = svm.score(X_test, y_test)

    # print test accuaracy
    print(train_acc, test_acc)
    # store svm coefs in dictionary
    concept_dict[concept] = (svm.coef_, train_acc, test_acc)  

airplane
air_conditioner
apron
arm
armchair
ashcan
awning
back
back_pillow
bag
balcony
bannister
base
basket
bathroom_s
bathtub
beak
bed
bedclothes
bedroom_s
bench
bicycle
bird
blackness
blind
blotchy
blueness
blurriness
board
body
book
bookcase
bottle
bowl
box
brick
bridge
bucket
building
bumper
bus
bush
cabinet
can
candlestick
canopy
cap
car
cardboard
carpet
cat
ceiling
ceramic
chain_wheel
chair
chandelier
chest_of_drawers
chimney
clock
coach
coffee_table
column
computer
concrete
counter
countertop
cow
cup
curtain
cushion
desk
dining_room_s
dog
door
doorframe
door_frame
double_door
drawer
drinking_glass
ear
earth
engine
exhaust_hood
eye
eyebrow
fabric
fan
faucet
fence
field
figurine
fireplace
flag
floor
flower
flowerpot
fluorescent
food
foot
footboard
frame
glass
granite
grass
greenness
ground
hair
hand
handle
handle_bar
head
headboard
headlight
hill
horse
house
inside_arm
jar
keyboard
knob
laminate
lamp
leather
leg
lid
light
loudspeaker
manhole
metal
microwave
minibike
mirror
motorb

In [35]:
concept_dict_coef = {}
for key in concept_dict:
    print(key, end=' ')
    svm, train_acc, test_acc = concept_dict[key]
    try:
        concept_dict_coef[key] = svm.coef_, train_acc, test_acc
    except:
        concept_dict_coef[key] = svm, train_acc, test_acc        

airplane air_conditioner apron arm armchair ashcan awning back back_pillow bag balcony bannister base basket bathroom_s bathtub beak bed bedclothes bedroom_s bench bicycle bird blackness blind blotchy blueness blurriness board body book bookcase bottle bowl box brick bridge bucket building bumper bus bush cabinet can candlestick canopy cap car cardboard carpet cat ceiling ceramic chain_wheel chair chandelier chest_of_drawers chimney clock coach coffee_table column computer concrete counter countertop cow cup curtain cushion desk dining_room_s dog door doorframe door_frame double_door drawer drinking_glass ear earth engine exhaust_hood eye eyebrow fabric fan faucet fence field figurine fireplace flag floor flower flowerpot redness tree water greenness polka_dots stripes fluorescent food foot footboard frame glass granite grass ground hair hand handle handle_bar head headboard headlight hill horse house inside_arm jar keyboard knob laminate lamp leather leg lid light loudspeaker manhole 

In [33]:
len(concept_dict)

170

In [37]:
pickle.dump(concept_dict_coef, open('sq_concepts_170.pkl', 'wb'))

In [38]:
for key in concept_dict_coef:
    coefs, train_acc, val_acc = concept_dict_coef[key]
    if val_acc < 0.7:
        print(key, val_acc)

back 0.64
beak 0.6
bed 0.6886792452830188
chair 0.68
figurine 0.6511627906976745
laminate 0.68
sofa 0.68


In [39]:
for key in concept_dict_coef:
    coefs, train_acc, val_acc = concept_dict_coef[key]
    if val_acc < 0.67:
        print(key, val_acc)

back 0.64
beak 0.6
figurine 0.6511627906976745
